In [1]:
from transformers import pipeline
large_masker = pipeline('fill-mask', model = 'bert-large-uncased-whole-word-masking')
base_masker = pipeline('fill-mask', model='bert-base-cased')

Downloading:   0%|          | 0.00/230 [00:00<?, ?B/s]

Couldn't reach server at 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-large-uncased-whole-word-masking-modelcard.json' to download model card file.
Creating an empty model card.


Downloading:   0%|          | 0.00/230 [00:00<?, ?B/s]

Couldn't reach server at 'https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-cased-modelcard.json' to download model card file.
Creating an empty model card.


In [15]:
import PyPDF2 as py
import nltk.data as nl
from tika import parser
import os
import spacy
import en_core_web_sm
import re
from spacy.lang.en.stop_words import STOP_WORDS
from collections import Counter
import transformers
from tika import parser
import bs4

In [41]:
def numPages(file_name):
    file_name = file_name + '.pdf'
    file_path = os.getcwd() + '\\' + file_name
    pdf = open(file_path, 'rb')
    pdf_read = py.PdfFileReader(pdf)
    print(pdf_read.getNumPages())
    
def displayPDF(file_name, begin, end):
    len = end - begin
    #print(file_name + '.pdf')
    file_name = file_name + '.pdf'
    file_path = os.getcwd() + '\\' + file_name
    with open(file_path, mode = 'rb') as f:
        for i in range(len + 1):
            reader = py.PdfFileReader(f)
            page = reader.getPage(begin + i - 1)
            print(page.extractText())
            
def displayAll(file_name):
    file_name = file_name + '.pdf'
    file_path = os.getcwd() + '\\' + file_name
    pdf = open(file_path, 'rb')
    pdf_read = py.PdfFileReader(pdf)
    i = 0
    while i < pdf_read.getNumPages():
        page = pdf_read.getPage(i)
        text = page.extractText()
        print(text)
        i = i + 1

def ExtractText(file_name):
    raw = parser.from_file(os.getcwd() + '\\' + file_name + '.pdf',xmlContent=True)
    return raw['content'][45:] 

def ExtractNouns(name):
    nlp = spacy.load("en_core_web_sm")
    pdf = nlp(name)
    nouns = []
    for token in pdf:
        if token.pos_ == "NOUN":
            nouns.append(token)
    return nouns

def ExtractVerbs(name):
    nlp = spacy.load("en_core_web_sm")
    pdf = nlp(name)
    verbs = []
    for token in pdf:
        if token.pos_ == "VERB":
            verbs.append(token)
    return verbs

def ExtractAdjectives(name):
    nlp = spacy.load("en_core_web_sm")
    pdf = nlp(name)
    adjectives = []
    for token in pdf:
        if token.pos_ == "ADJECTIVES":
            adjectives.append(token)
    return adjectives

def toLowerCase(list):
    new_list = []
    for word in list:
        word = word.lower()
        new_list.append(word)
    return new_list

def clean(input_list):
    return [w for w in [str(tok) for tok in input_list] if len(w) > 0 and not w.isdigit()]
    

def Filter(list):
    list_clean = clean(list)
    new_list = []
    list_lower = toLowerCase(list_clean)
    jargon = ExtractText('PoliticalJargon')
    jargonNouns = ExtractNouns(jargon)
    jargonNouns_clean = clean(jargonNouns)
    jargonNouns_lower = toLowerCase(jargonNouns_clean)
    for noun in list_lower:
        for political_noun in jargonNouns_lower:
            if noun == political_noun:
                new_list.append(noun)    
    return set(new_list)
                
def wordsRemoved(list1, list2):
    s1 = set(list1)
    s2 = set(list2)
    return s2 - s1

def getJargoninMap(name):
    PDF_list = list(name.split("."))
    
    PDF_verbs = ExtractVerbs(name)
    PDF_filtered_verbs = Filter(PDF_verbs)
    verbs_dict = {}
    for word in PDF_filtered_verbs:
        verbs_dict[word] = 'Verb'
    nouns_dict = {}
    PDF_nouns = ExtractNouns(name)
    PDF_filtered_nouns = Filter(PDF_nouns)
    for word in PDF_filtered_nouns:
        nouns_dict[word] = 'Noun'
    jargon = {**verbs_dict, **nouns_dict}
    if '-' in jargon:
        del jargon['-']
    return jargon
    
def paragraphs(text):
  exampleSoup = bs4.BeautifulSoup(text, 'html.parser')
  pages = exampleSoup.select("p")
  paragraphs = []
  i = 0
  for item in pages:

    if (len(item.getText()) > 15 and item.getText().strip().endswith(".")==True):
      if(item.getText().split(".")[0].istitle()== False):
        k = item.getText().replace("\n", " ")
        paragraphs.append(k)

  return paragraphs



In [42]:
Capitalism1 = ExtractText('Capitalism1')

In [43]:
x = paragraphs(Capitalism1)

In [44]:
print(x)

[]


In [45]:
jargon = getJargoninMap(Capitalism1)

In [46]:
type(jargon)

dict

In [47]:
print(jargon)

{'run': 'Noun', 'monopoly': 'Verb', 'benefit': 'Verb', 'path': 'Noun', 'class': 'Noun', 'forces': 'Noun', 'nationalism': 'Noun', 'framework': 'Noun', 'inflation': 'Noun', 'exchange': 'Noun', 'trade': 'Noun', 'profit': 'Noun', 'capital': 'Noun', 'opinion': 'Noun', 'account': 'Noun', 'risk': 'Noun', 'regulations': 'Noun', 'centre': 'Noun', 'order': 'Noun', 'creation': 'Noun', 'system': 'Noun', 'term': 'Noun', 'tendency': 'Noun', 'figure': 'Noun', 'wage': 'Noun', 'utility': 'Noun', 'economies': 'Noun', 'wages': 'Noun', 'consumer': 'Noun', 'way': 'Noun', 'advantage': 'Noun', 'problem': 'Noun', 'growth': 'Noun', 'dichotomy': 'Noun', 'power': 'Noun', 'process': 'Noun', 'demand': 'Noun', 'factor': 'Noun', 'workers': 'Noun', 'cost': 'Noun', 'savings': 'Noun', 'money': 'Noun', 'tradition': 'Noun', 'business': 'Noun', 'globalizations': 'Noun', 'print': 'Noun', 'unemployment': 'Noun', 'advance': 'Noun', 'economy': 'Noun', 'price': 'Noun', 'theory': 'Noun', 'people': 'Noun', 'output': 'Noun', 'rul

In [48]:
## Everything Necessary is above. Everything below is just for further analysis and not necessary

In [49]:
CapitatlismVerbs = ExtractVerbs(Capitalism1)
print(CapitatlismVerbs)

[led, can, found, https://www.tandfonline.com/loi/yjcr20, cite, link, Published, Published, Submit, Citing, citing, https://www.tandfonline.com/doi/citedby/10.1080/14767430.2017.1332807#tabModule, https://www.tandfonline.com/doi/citedby/10.1080/14767430.2017.1332807#tabModule, can, specified, put, know, offers, seems, offered, glimpse, attempt, may, weaving, overlap, viewed, turning, rendered, will, returning, achieve, provide, understanding, provides, provides, covers, Given, describing, assessing, indicates, analyse, understand, driven, regulates, generate, recurring, cending, given, sets, turns, seek, suffer, surviving, go, bought, may, make, attract, trying, overcome, involving, seeks, achieve, REALISM, relying, explores, Consider, assume, maximizing, optimizing, explain, respond, constrained, suffices, arise, dictate, can, intervene, breaking, forms, grounded, maintains, determine, run, grounded, comes, frames, developed, appears, improve, seems, emerges, evolve, changes, managed,

In [50]:
filteredVerbs = Filter(CapitatlismVerbs)
print(filteredVerbs)

{'run', 'monopoly', 'benefit'}


In [51]:
verbsRemoved = wordsRemoved(filteredVerbs, toLowerCase(clean(CapitatlismVerbs)))
print(verbsRemoved)

{'tries', 'use', 'closed', 'ask', 'resisted', 'explain', 'citing', 'put', 'prevail', 'born', 'approach', 'frames', 'underlying', 'accept', 'criticizes', 'serve', 'acknowledges', 'consists', 'specified', 'led', 'developed', 'characterize', 'agree', 'point', 'construed', 'arise', 'understanding', 'undermine', 'doing', 'make', 'optimizing', 'affecting', 'heading', 'regulates', 'emerging', 'give', 'describing', 'reconstructing', 'estimated', 'see', 'given', 'trying', 'say', 'https://www.tandfonline.com/loi/yjcr20', 'covers', 'take', 'might', 'go', 'expressed', 'define', 'relying', 'result', 'submit', 'causes', 'overcoming', 'reflect', 'acknowledge', 'wants', 'doubled', 'close', 'mentioning', 'leads', 'increasing', 'surviving', 'provided', 'evolve', 'provide', 'turns', 'indicates', 'analysed', 'maximizing', 'constituted', 'indicate', 'leaves', 'altered', 'follow', 'swept', 'would', 'adjusting', 'appears', 'comes', 'require', 'decrease', 'contrasting', 'https://www.tandfonline.com/doi/citedb

In [52]:

CapitatlismNouns = ExtractNouns(Capitalism1)
print(CapitatlismNouns)

[access, use, Print, homepage, Capitalism, Competition, Conflict, Crisis, article, Capitalism, Competition, Conflict, article, Author(s, trading, article, journal, Article, views, articles, data, articles, articles, advance, risk, possibility, transformation, engagement, art, theory, case, study, engagement, happening, truth, engagement, possibility, transformation, appraisal, vision, curriculum, education, chimera, constellation, teacher, student, curricu-, lum, matter, book, thing, interpretation, tradition, concern, discussion, methodology, journey, set, meanderings, pathways, circle, re, -, mapping, land-, scape, university, anthropology, gaze, homeland, trading, Capitalism, Competition, Conflict, book, time, time, attempt, objective, framework, economies, source, discussions, theory, economy, book, framework, theory, ambition, scope, book, review, undertaking, whole, title, book, ambition, capitalism, system, Capitalism, profit, motive, supply, demand, capitalism, forces, patterns

In [53]:
filteredNouns = Filter(CapitatlismNouns)
print(filteredNouns)

{'path', 'class', 'forces', 'nationalism', 'framework', 'inflation', 'exchange', 'trade', 'profit', 'capital', 'opinion', 'account', 'risk', 'regulations', 'centre', 'order', 'creation', 'system', 'term', 'tendency', 'figure', 'wage', 'utility', 'economies', 'wages', 'consumer', 'way', 'advantage', 'problem', 'growth', 'dichotomy', 'power', 'process', 'demand', 'factor', 'workers', 'cost', 'savings', 'money', 'tradition', 'business', 'globalizations', 'print', 'unemployment', 'advance', 'economy', 'price', 'theory', 'people', 'output', 'rules', 'development', 'hand', 'globalization', 'economics', 'value', 'policy', 'balance', 'peace', 'labour', 'entry', 'rate', 'crisis', 'surge', 'firms', 'employment', 'socialism', '-', 'policies', 'progress', 'capacity', 'run', 'markets', 'prices', 'population', 'metaphors', 'capitalism', 'goods', 'time', 'state', 'costs', 'competition', 'agency', 'freedom', 'title', 'model', 'issue', 'methods', 'motive', 'nation', 'politics', 'wars', 'terms', 'return

In [54]:
nounsRemoved = wordsRemoved(filteredNouns, toLowerCase(clean(CapitatlismNouns)))
print(nounsRemoved)

{'era', 'particularities', 'dency', 'alism', 'use', 'dard', 'ontology', 'evidence', 'speakers', '2010s', 'agreement', 'criti-', 'space', 'learning', 'localities', 'end', 'orthodoxy', 'lation', 'losses', 'chapter', 'reviews', 'systems', 'approach', 'amount', 'teacher', 'institutions', 'century', 'crises', 'average', 'truth', 'turn', 'fall', 'principles', 'formu-', 'laws', 'bit', 'thinker', 'alienation', 'point', 'while', 'turbulence', 'understanding', 'accounts', 'metaphysics', 'pessimist', 'downs', 'mechanisms', 'unevenness', 'passages', 'container', 'appraisal', 'centuries', 'booms', 'stage', 'gdp', 'grounds', 'decreases', 'thing', 'references', 'say', 'subjectivity', 'conclusion', 'pessimism', 'imbalances', 'variables', 'picture', 'capital-', 'ups', 'accumulation', 'aspect', 'journey', 'curriculum', 'economists', 'processes', 'causes', 'approaches', 'self', 'story', 'winners', 'medium', 'viewpoint', 'movement', 'education', 'handbook', 'explanations', 'clarifies', 'gravitation', '182

In [55]:
JargonWords = filteredNouns.union(filteredVerbs)
print(JargonWords)

{'rules', 'development', 'path', 'class', 'forces', 'hand', 'globalization', 'economics', 'nationalism', 'value', 'policy', 'balance', 'peace', 'framework', 'labour', 'inflation', 'exchange', 'entry', 'rate', 'trade', 'crisis', 'surge', 'firms', 'employment', 'socialism', 'profit', '-', 'policies', 'capital', 'opinion', 'progress', 'capacity', 'run', 'account', 'markets', 'risk', 'regulations', 'centre', 'order', 'creation', 'system', 'prices', 'term', 'tendency', 'figure', 'population', 'metaphors', 'capitalism', 'goods', 'time', 'wage', 'utility', 'state', 'costs', 'economies', 'wages', 'consumer', 'way', 'advantage', 'problem', 'competition', 'growth', 'dichotomy', 'power', 'agency', 'process', 'demand', 'factor', 'workers', 'cost', 'freedom', 'savings', 'title', 'model', 'money', 'tradition', 'benefit', 'issue', 'methods', 'motive', 'nation', 'business', 'politics', 'globalizations', 'wars', 'terms', 'return', 'print', 'hypothesis', 'unemployment', 'supply', 'barriers', 'democracy'

In [56]:
print(len(filteredNouns))
print(len(filteredVerbs))
print(len(jargon))


107
3
109


In [57]:
x = large_masker("Hello I'm a [MASK] model.")

In [58]:
print(x)

[{'sequence': "[CLS] hello i'm a fashion model. [SEP]", 'score': 0.16184712946414948, 'token': 4827}, {'sequence': "[CLS] hello i'm a cover model. [SEP]", 'score': 0.10551388561725616, 'token': 3104}, {'sequence': "[CLS] hello i'm a male model. [SEP]", 'score': 0.08243890106678009, 'token': 3287}, {'sequence': "[CLS] hello i'm a new model. [SEP]", 'score': 0.0381552018225193, 'token': 2047}, {'sequence': "[CLS] hello i'm a top model. [SEP]", 'score': 0.037600647658109665, 'token': 2327}]


In [59]:
y = base_masker("Hello I'm a [MASK] model.")

In [60]:
print(y)

[{'sequence': "[CLS] Hello I'm a new model. [SEP]", 'score': 0.09941332787275314, 'token': 1207}, {'sequence': "[CLS] Hello I'm a fashion model. [SEP]", 'score': 0.08853159099817276, 'token': 4633}, {'sequence': "[CLS] Hello I'm a male model. [SEP]", 'score': 0.053984664380550385, 'token': 2581}, {'sequence': "[CLS] Hello I'm a professional model. [SEP]", 'score': 0.04418348893523216, 'token': 1848}, {'sequence': "[CLS] Hello I'm a former model. [SEP]", 'score': 0.03014710545539856, 'token': 1393}]


In [61]:
Capitalism1_list = list(Capitalism1.split("."))
Capitalism1_list


['led\n\n\nFull Terms & Conditions of access and use can be found at\nhttps://www',
 'tandfonline',
 'com/action/journalInformation?journalCode=yjcr20\n\nJournal of Critical Realism\n\nISSN: 1476-7430 (Print) 1572-5138 (Online) Journal homepage: https://www',
 'tandfonline',
 'com/loi/yjcr20\n\nCapitalism: Competition, Conflict, Crisis\n\nHeikki Patomäki\n\nTo cite this article: Heikki Patomäki (2017) Capitalism: Competition, Conflict, Crisis, Journal of\nCritical Realism, 16:5, 537-543, DOI: 10',
 '1080/14767430',
 '2017',
 '1332807\n\nTo link to this article:  https://doi',
 'org/10',
 '1080/14767430',
 '2017',
 '1332807\n\n© 2017 The Author(s)',
 ' Published by Informa\nUK Limited, trading as Taylor & Francis\nGroup\n\nPublished online: 01 Jun 2017',
 '\n\nSubmit your article to this journal \n\nArticle views: 11052\n\nView related articles \n\nView Crossmark data\n\nCiting articles: 1 View citing articles \n\nhttps://www',
 'tandfonline',
 'com/action/journalInformation?journalCode

In [80]:
Capitalism1_list[80]

' In capitalism, many forces together generate recurring patterns trans-\ncending historical and regional particularities: the profit motive being the most powerful of\nthese forces'

In [62]:
Capitalism1_list[45]

'1332807&domain=pdf&date_stamp=2017-06-01\nhttp://crossmark'

In [63]:
print(jargon)

{'run': 'Noun', 'monopoly': 'Verb', 'benefit': 'Verb', 'path': 'Noun', 'class': 'Noun', 'forces': 'Noun', 'nationalism': 'Noun', 'framework': 'Noun', 'inflation': 'Noun', 'exchange': 'Noun', 'trade': 'Noun', 'profit': 'Noun', 'capital': 'Noun', 'opinion': 'Noun', 'account': 'Noun', 'risk': 'Noun', 'regulations': 'Noun', 'centre': 'Noun', 'order': 'Noun', 'creation': 'Noun', 'system': 'Noun', 'term': 'Noun', 'tendency': 'Noun', 'figure': 'Noun', 'wage': 'Noun', 'utility': 'Noun', 'economies': 'Noun', 'wages': 'Noun', 'consumer': 'Noun', 'way': 'Noun', 'advantage': 'Noun', 'problem': 'Noun', 'growth': 'Noun', 'dichotomy': 'Noun', 'power': 'Noun', 'process': 'Noun', 'demand': 'Noun', 'factor': 'Noun', 'workers': 'Noun', 'cost': 'Noun', 'savings': 'Noun', 'money': 'Noun', 'tradition': 'Noun', 'business': 'Noun', 'globalizations': 'Noun', 'print': 'Noun', 'unemployment': 'Noun', 'advance': 'Noun', 'economy': 'Noun', 'price': 'Noun', 'theory': 'Noun', 'people': 'Noun', 'output': 'Noun', 'rul

In [68]:
del jargon['-']

In [70]:
print(jargon)

{'run': 'Noun', 'monopoly': 'Verb', 'benefit': 'Verb', 'path': 'Noun', 'class': 'Noun', 'forces': 'Noun', 'nationalism': 'Noun', 'framework': 'Noun', 'inflation': 'Noun', 'exchange': 'Noun', 'trade': 'Noun', 'profit': 'Noun', 'capital': 'Noun', 'opinion': 'Noun', 'account': 'Noun', 'risk': 'Noun', 'regulations': 'Noun', 'centre': 'Noun', 'order': 'Noun', 'creation': 'Noun', 'system': 'Noun', 'term': 'Noun', 'tendency': 'Noun', 'figure': 'Noun', 'wage': 'Noun', 'utility': 'Noun', 'economies': 'Noun', 'wages': 'Noun', 'consumer': 'Noun', 'way': 'Noun', 'advantage': 'Noun', 'problem': 'Noun', 'growth': 'Noun', 'dichotomy': 'Noun', 'power': 'Noun', 'process': 'Noun', 'demand': 'Noun', 'factor': 'Noun', 'workers': 'Noun', 'cost': 'Noun', 'savings': 'Noun', 'money': 'Noun', 'tradition': 'Noun', 'business': 'Noun', 'globalizations': 'Noun', 'print': 'Noun', 'unemployment': 'Noun', 'advance': 'Noun', 'economy': 'Noun', 'price': 'Noun', 'theory': 'Noun', 'people': 'Noun', 'output': 'Noun', 'rul

In [89]:
indexs = []
for sentence in Capitalism1_list:
    for word in sentence.split(" "):
        if word in jargon:
            #print(word)
            indexs.append(Capitalism1_list.index(sentence))
         
       # index = animals.index('dog')

In [90]:
print(indexs)

[12, 61, 68, 74, 74, 75, 75, 76, 76, 78, 78, 79, 79, 79, 80, 80, 80, 80, 81, 81, 81, 82, 82, 83, 84, 85, 85, 85, 86, 87, 89, 97, 98, 98, 98, 99, 100, 100, 102, 103, 104, 104, 105, 106, 106, 106, 107, 107, 107, 108, 109, 114, 115, 115, 115, 115, 115, 115, 117, 119, 119, 119, 122, 125, 126, 126, 126, 128, 128, 128, 129, 129, 131, 131, 131, 132, 133, 133, 137, 138, 139, 139, 140, 140, 141, 141, 142, 142, 142, 143, 144, 144, 144, 144, 145, 145, 145, 147, 147, 147, 148, 148, 148, 148, 149, 149, 150, 151, 152, 152, 152, 153, 153, 153, 153, 153, 154, 154, 155, 156, 156, 156, 157, 157, 157, 158, 158, 159, 161, 162, 163, 164, 169, 169, 173, 173, 173, 173, 174, 174, 174, 175, 178, 179, 180, 181, 181, 182, 186, 187, 187, 188, 193, 194, 196, 196, 196, 196, 197, 198, 199, 201, 203, 205, 205, 206, 206, 209, 210, 210, 211, 213, 215, 215, 216, 216, 217, 218, 219, 219, 219, 219, 220, 221, 221, 223, 224, 224, 224, 224, 224, 224, 225, 226, 226, 227, 227, 228, 229, 230, 232, 291, 292, 294]


In [ ]:
for key in jargon:
    if

In [92]:
Capitalism1_list[224]

' He argues against the Ricardian theory of free\ntrade in terms of his theory of real competition, which implies that success comes with absolute\ncost advantage at the level of firms and may result in systematic and persistent imbalances in\nthe world economy (Chapter 11)'

In [99]:
print(Capitalism1_list[224].split(" "))

['', 'He', 'argues', 'against', 'the', 'Ricardian', 'theory', 'of', 'free\ntrade', 'in', 'terms', 'of', 'his', 'theory', 'of', 'real', 'competition,', 'which', 'implies', 'that', 'success', 'comes', 'with', 'absolute\ncost', 'advantage', 'at', 'the', 'level', 'of', 'firms', 'and', 'may', 'result', 'in', 'systematic', 'and', 'persistent', 'imbalances', 'in\nthe', 'world', 'economy', '(Chapter', '11)']


In [105]:
e = Capitalism1_list[224].split(" ")

In [104]:
x = Capitalism1_list[224].split(" ").index("competition,")

AttributeError: 'list' object has no attribute 'strip'

In [103]:
print(x)

16
